In [21]:
import numpy as np
import pandas as pd
from siuba import *
from calitp import *
import intake
import shared_utils

import data_prep
import _utils

In [24]:
df = pd.read_parquet('BC_GTFS_NTD.parquet')

In [25]:
df.sample()

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,...,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,itp_id,Is_Agency_In_BC_Only_1_means_Yes,GTFS,fleet_size
766,2021,Section 5311,TBD | 0021000227,2021,County of Shasta Department of Public Works,BCG0002920,Operating Assistance Sliding Scale,300902,None,206903.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0,Ok,Small


In [26]:
df>>filter(_.organization_name=="Madera County")>>count(_.ali)

,ali,n
0,111215,2
1,111304,5
2,117A00,5
3,300902,6


In [27]:
df>>group_by(_.organization_name)>>summarize(n = _.activebalance.mean())>>filter(_.n<=0)

,organization_name,n
5,City of Arcata,0.000000
14,"City of Eureka, dba: Eureka Transit Service",0.000000
15,City of Fairfield,0.000000
44,"Greyhound Lines, Inc.",-39110.769000
48,Kings County Area Public Transit Agency,-3894.727273


In [28]:
column_names = ['_16_20', '_21_25', '_26_30', '_31_60']
df['_16plus']= df[column_names].sum(axis=1)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798 entries, 0 to 797
Data columns (total 45 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   grant_fiscal_year                   798 non-null    int64  
 1   funding_program                     798 non-null    object 
 2   grant_number                        798 non-null    object 
 3   project_year                        798 non-null    int64  
 4   organization_name                   798 non-null    object 
 5   upin                                798 non-null    object 
 6   description                         798 non-null    object 
 7   ali                                 798 non-null    object 
 8   contract_number                     682 non-null    object 
 9   allocationamount                    798 non-null    float64
 10  encumbered_amount                   798 non-null    float64
 11  expendedamount                      798 non-n

In [30]:
df_agg = df.groupby(['organization_name',
                     'reporter_type',
                     'GTFS',
                     'fleet_size',
                     'ntd_id',
                     'itp_id']).agg({'allocationamount':'mean',
                                         'encumbered_amount':'mean',
                                         'expendedamount':'mean',
                                         'encumbered_amount':'mean',
                                         'activebalance':'mean',
                                         'closedoutbalance':'mean',
                                         'expendedamount':'mean',
                                         'total_vehicles':'max',
                                         'average_age_of_fleet__in_years_':'max',
                                         'average_lifetime_miles_per_vehicle':'max',
                                         'Automobiles':'max',
                                         'Bus':'max',
                                         'Other':'max',
                                         'Train':'max',
                                         'Van':'max',
                                         'automobiles_door':'max',
                                         'bus_doors':'max',
                                         'van_doors':'max',
                                         'train_doors':'max',
                                         'doors_sum':'max',
                                         '_0_9':'max',
                                         '_10_12':'max',
                                         '_13_15':'max',
                                         '_16_20':'max',
                                         '_21_25':'max',
                                         '_26_30':'max',
                                         '_31_60':'max',
                                         '_16plus':'max',
                                         '_60plus':'max'})


df_agg= df_agg.reset_index()

df_agg.head()


,organization_name,reporter_type,GTFS,fleet_size,ntd_id,itp_id,allocationamount,encumbered_amount,expendedamount,activebalance,...,doors_sum,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_16plus,_60plus
0,Amador Regional Transit System,Rural Reporter,Ok,Small,9R02-91000,11.0,189817.375000,159889.250000,133498.000000,56319.375000,...,21.0,18.0,3.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0
1,Butte County Association of Governments,Full Reporter,Ok,Medium,90208,47.0,490548.916667,401737.500000,363577.065000,126971.851667,...,82.0,44.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Calaveras Transit Agency,Rural Reporter,None,Small,9R02-99442,0.0,187646.000000,212771.888889,120177.108889,67468.891111,...,11.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Central Contra Costa Transit Authority, dba: C...",Full Reporter,None,Large,90078,0.0,50412.000000,50412.000000,0.000000,50412.000000,...,312.0,160.0,43.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0
4,City of Arcata,Rural Reporter,Ok,Small,9R02-91018,18.0,82111.428571,143540.000000,82111.428571,0.000000,...,12.0,5.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Adding Districts

In [48]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/5311 /"
File_Organization =  "organizations-AllOrganizations_1.csv"
organizations =  pd.read_csv(f'{GCS_FILE_PATH}{File_Organization}')
test2 = to_snakecase(organizations)

In [51]:
test2.sample()

,name,organization_type,roles,record_creation_time,ntp_id,itp_id,opm_id_drmt,dotid,brand,alias,...,service_availability_category__from_mobility_services_managed_,#_fixed_route_or_deviated_fixed_route_services,#_fixed_route_or_deviated_fixed_route_service_w__static_gtfs,#_services_with_missing_static_feed_for_fixed_route_or_deviated_fixed_route,county_geography,county_geography_3,gtfs_static_status,gtfs_realtime_status,service_availability,service___component
258,City of National City,City/Town,NaN,1/10/2022 9:59pm,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,NaN,NaN,Static OK,RT Incomplete,NaN,NaN


In [88]:
test2 = test2>>select(_.name,
                      # _.ntp_id,
                      # _.itp_id,
                      _.caltrans_district,
                      _.gtfs_static_status,
                      _.gtfs_realtime_status)    

In [89]:
test2

,name,caltrans_district,gtfs_static_status,gtfs_realtime_status
0,A-Paratransit,NaN,Static OK,RT Incomplete
1,ABC Shuttle,NaN,Static OK,RT Incomplete
2,Able Inc.,NaN,Static OK,RT Incomplete
3,Abrazar Inc.,NaN,Static OK,RT Incomplete
4,Access Services,07 - Los Angeles,Static OK,RT Incomplete
...,...,...,...,...
815,Yosemite Area Regional Transportation System,10 - Stockton,Static OK,RT Incomplete
816,Yuba-Sutter Transit Authority,03 - Marysville,Static OK,RT Incomplete
817,Yuma County Intergovernmental Public Transport...,11 - San Diego,Static OK,RT Incomplete
818,Yurok Tribe,01 - Eureka,Static OK,RT Incomplete


In [92]:
#merging on organization name because itp_id does matches wrong agencies with no itp_id

In [91]:
test_merge3 = pd.merge(df_agg, test2, how='left', left_on='organization_name', right_on='name')
test_merge3

,organization_name,reporter_type,GTFS,fleet_size,ntd_id,itp_id,allocationamount,encumbered_amount,expendedamount,activebalance,...,_16_20,_21_25,_26_30,_31_60,_16plus,_60plus,name,caltrans_district,gtfs_static_status,gtfs_realtime_status
0,Amador Regional Transit System,Rural Reporter,Ok,Small,9R02-91000,11.0,189817.375000,159889.250000,133498.000000,56319.375000,...,1.0,1.0,0.0,0.0,2.0,0.0,Amador Regional Transit System,10 - Stockton,Static OK,RT Incomplete
1,Butte County Association of Governments,Full Reporter,Ok,Medium,90208,47.0,490548.916667,401737.500000,363577.065000,126971.851667,...,0.0,0.0,0.0,0.0,0.0,0.0,Butte County Association of Governments,03 - Marysville,Static OK,RT Incomplete
2,Calaveras Transit Agency,Rural Reporter,None,Small,9R02-99442,0.0,187646.000000,212771.888889,120177.108889,67468.891111,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,"Central Contra Costa Transit Authority, dba: C...",Full Reporter,None,Large,90078,0.0,50412.000000,50412.000000,0.000000,50412.000000,...,1.0,1.0,0.0,0.0,2.0,0.0,NaN,NaN,NaN,NaN
4,City of Arcata,Rural Reporter,Ok,Small,9R02-91018,18.0,82111.428571,143540.000000,82111.428571,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,City of Arcata,01 - Eureka,Static OK,RT OK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,Tuolumne County Transit,Rural Reporter,Needed,Medium,9R02-91057,482.0,316718.166667,258717.833333,258717.833333,58000.333333,...,1.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN
74,Victor Valley Transit Authority,Full Reporter,Ok,Large,90148,360.0,665100.166667,542216.500000,542216.500000,122883.666667,...,1.0,0.0,0.0,0.0,1.0,0.0,Victor Valley Transit Authority,08 - San Bernardino,Static OK,RT OK
75,Yolo County Transportation District,Full Reporter,Ok,Large,90090,372.0,140163.166667,114495.000000,114495.000000,25668.166667,...,8.0,8.0,0.0,0.0,16.0,0.0,Yolo County Transportation District,03 - Marysville,Static OK,RT Incomplete
76,Yosemite Area Regional Transportation System,Rural Reporter,Ok,Small,9R02-91070,374.0,225648.250000,200648.250000,188148.250000,37500.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,Yosemite Area Regional Transportation System,10 - Stockton,Static OK,RT Incomplete


## To Parquet

In [94]:
#test_merge3.to_parquet("5311_agg.parquet")